# Topics and Transformations

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Creating the Corpus

In [2]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2023-04-04 11:20:19,132 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-04-04 11:20:19,134 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2023-04-04 11:20:19,136 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-04-04T11:20:19.136846', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


# Creating a transformation

In [3]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2023-04-04 11:20:36,526 : INFO : collecting document frequencies
2023-04-04 11:20:36,530 : INFO : PROGRESS: processing document #0
2023-04-04 11:20:36,532 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2023-04-04T11:20:36.532387', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'initialize'}


# Transforming vectors

In [4]:
# Transformation on 1 vector
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [5]:
# Transformation on all vectors
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [6]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2023-04-04 11:22:23,403 : INFO : using serial LSI version on this node
2023-04-04 11:22:23,413 : INFO : updating model with new documents
2023-04-04 11:22:23,418 : INFO : preparing a new chunk of documents
2023-04-04 11:22:23,421 : INFO : using 100 extra samples and 2 power iterations
2023-04-04 11:22:23,423 : INFO : 1st phase: constructing (12, 102) action matrix
2023-04-04 11:22:23,429 : INFO : orthonormalizing (12, 102) action matrix
2023-04-04 11:22:23,433 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-04-04 11:22:23,436 : INFO : computing the final decomposition
2023-04-04 11:22:23,439 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2023-04-04 11:22:23,484 : INFO : processed documents up to #9
2023-04-04 11:22:23,490 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-04-04 11:22:23,493 : INFO : topic #

In [7]:
lsi_model.print_topics(2)

2023-04-04 11:22:34,951 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-04-04 11:22:34,954 : INFO : topic #1(1.476): 0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"')]

In [8]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090823), (1, 0.5200703306361845)] Human machine interface for lab abc computer applications
[(0, 0.19667592859143246), (1, 0.760956316770003)] A survey of user opinion of computer system response time
[(0, 0.08992639972447164), (1, 0.72418606267525)] The EPS user interface management system
[(0, 0.07585847652178741), (1, 0.6320551586003424)] System and human system engineering testing of EPS
[(0, 0.10150299184980752), (1, 0.5737308483002942)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378289), (1, -0.16115180214026456)] The generation of random binary unordered trees
[(0, 0.8774787673119806), (1, -0.16758906864660217)] The intersection graph of paths in trees
[(0, 0.9098624686818554), (1, -0.14086553628719833)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569278), (1, 0.05392907566388862)] Graph minors A survey


In [9]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

loaded_lsi_model = models.LsiModel.load(tmp.name)

os.unlink(tmp.name)

2023-04-04 11:23:18,989 : INFO : Projection lifecycle event {'fname_or_handle': 'C:\\Users\\AxelArcidiaco\\AppData\\Local\\Temp\\model-xpqk9fka.lsi.projection', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-04-04T11:23:18.989591', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
2023-04-04 11:23:18,994 : INFO : saved C:\Users\AxelArcidiaco\AppData\Local\Temp\model-xpqk9fka.lsi.projection
2023-04-04 11:23:18,997 : INFO : LsiModel lifecycle event {'fname_or_handle': 'C:\\Users\\AxelArcidiaco\\AppData\\Local\\Temp\\model-xpqk9fka.lsi', 'separately': 'None', 'sep_limit': 10485760, 'ignore': ['projection', 'dispatcher'], 'datetime': '2023-04-04T11:23:18.997148', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
2023-04-04 11:23:19,000 : INF

# Available transformations

In [14]:
# Term Frequency * Inverse Document Frequency, Tf-Idf
model = models.TfidfModel(corpus, normalize=True)
vector = model[corpus[0]] 

2023-04-04 11:26:27,725 : INFO : collecting document frequencies
2023-04-04 11:26:27,727 : INFO : PROGRESS: processing document #0
2023-04-04 11:26:27,730 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2023-04-04T11:26:27.730356', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'initialize'}


In [15]:
# Okapi Best Matching, Okapi BM25
model = models.OkapiBM25Model(corpus)

In [20]:
# Latent Semantic Indexing, LSI (or sometimes LSA) 
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

2023-04-04 11:29:13,280 : INFO : using serial LSI version on this node
2023-04-04 11:29:13,282 : INFO : updating model with new documents
2023-04-04 11:29:13,285 : INFO : preparing a new chunk of documents
2023-04-04 11:29:13,288 : INFO : using 100 extra samples and 2 power iterations
2023-04-04 11:29:13,292 : INFO : 1st phase: constructing (12, 400) action matrix
2023-04-04 11:29:13,296 : INFO : orthonormalizing (12, 400) action matrix
2023-04-04 11:29:13,300 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-04-04 11:29:13,301 : INFO : computing the final decomposition
2023-04-04 11:29:13,302 : INFO : keeping 9 factors (discarding 0.000% of energy spectrum)
2023-04-04 11:29:13,307 : INFO : processed documents up to #9
2023-04-04 11:29:13,308 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-04-04 11:29:13,310 : INFO : topic #1

In [22]:
# Random Projections, RP
model = models.RpModel(corpus_tfidf, num_topics=500)

2023-04-04 11:29:36,425 : INFO : no word id mapping provided; initializing from corpus, assuming identity
2023-04-04 11:29:36,427 : INFO : constructing (500, 12) random matrix
2023-04-04 11:29:36,432 : INFO : RpModel lifecycle event {'msg': 'created RpModel<num_terms=12, num_topics=500>', 'datetime': '2023-04-04T11:29:36.432798', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [23]:
# Latent Dirichlet Allocation, LDA
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

2023-04-04 11:29:47,749 : INFO : using symmetric alpha at 0.01
2023-04-04 11:29:47,752 : INFO : using symmetric eta at 0.01
2023-04-04 11:29:47,753 : INFO : using serial LDA version on this node
2023-04-04 11:29:47,758 : INFO : running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2023-04-04 11:29:47,759 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2023-04-04 11:29:47,774 : INFO : -185.885 per-word bound, 90561921191083318696447146088765140805030721748698398720.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2023-04-04 11:29:47,776 : INFO : PROGRESS: pass 0, at document #9/9
2023-04-04 11:29:47,783 : INFO : topic #41 (0.010): 0.083*"user" + 0.083*"system" + 0.083*"graph" + 0.083*"trees" 

In [24]:
# Hierarchical Dirichlet Process, HDP
model = models.HdpModel(corpus, id2word=dictionary)

2023-04-04 11:30:17,253 : INFO : (0, '0.154*trees + 0.151*human + 0.133*survey + 0.121*time + 0.117*system + 0.092*eps + 0.079*user + 0.065*graph + 0.039*interface + 0.024*response')
2023-04-04 11:30:17,255 : INFO : (1, '0.202*system + 0.191*user + 0.170*response + 0.106*trees + 0.084*human + 0.074*graph + 0.043*eps + 0.039*interface + 0.033*time + 0.030*minors')
2023-04-04 11:30:17,258 : INFO : (2, '0.285*minors + 0.256*human + 0.141*interface + 0.095*computer + 0.082*eps + 0.052*graph + 0.031*system + 0.023*response + 0.022*user + 0.011*survey')
2023-04-04 11:30:17,260 : INFO : (3, '0.166*computer + 0.119*graph + 0.116*minors + 0.096*human + 0.086*time + 0.083*trees + 0.070*system + 0.069*eps + 0.066*user + 0.058*interface')
2023-04-04 11:30:17,262 : INFO : (4, '0.214*survey + 0.189*computer + 0.138*response + 0.111*time + 0.090*system + 0.087*interface + 0.078*user + 0.036*trees + 0.027*human + 0.016*eps')
2023-04-04 11:30:17,265 : INFO : (5, '0.317*trees + 0.182*time + 0.126*human 